In [ ]:
import torch 
import torchvision.transforms as transforms
import torchvision

from Tools.train_test_model import train_test_model
from Tools.Visualize import Visualize
from ZFNet.ZFNet import ZFNet
from ZFNet.DeConv import DeConv
from ZFNet.Load_Model import Load_Model
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
transform = transforms.Compose(
    [
        transforms.Resize((224,224)),    
        transforms.ToTensor(),
        #transforms.Normalize(([0.1307]),([0.3081]))
    ]
    )

dataset = torchvision.datasets.CIFAR10(root="./Datasets",train=True, transform = transform,download=True)
train_set, val_set = torch.utils.data.random_split(dataset, [40000, 10000])
test_set = torchvision.datasets.CIFAR10(root="./Datasets",train=False,transform = transform,download=True)

In [ ]:
load_model = Load_Model()
network = ZFNet(10).cuda()
deconv = DeConv().cuda()
tool = train_test_model("ZFNet")
#tool.train_model(network,epoch=30,train_set=train_set,val_set=val_set,lr=[0.0001],batch_size=[20],num_worker=[4],saved_accuracy=80)

In [ ]:
import cv2
transform = transforms.Compose(
    [   transforms.ToPILImage(),
        transforms.Resize((224,224)),    
        transforms.ToTensor(),
        #transforms.Normalize(([0.1307]),([0.3081]))
    ]
    )

fig2 = plt.figure(figsize=(60,60))
#network.load_state_dict(torch.load("./ZFNet/saved_model/zfnet-1727-d010ddca.pth"))
load_model.load_model("./ZFNet/saved_models/ZFNet_with_accuracy=81.22.pth", network, deconv)
count = 0
#network.eval()
image = cv2.imread("./ZFNet/images/dog2.png")
image = torch.transpose(torch.from_numpy(image),1,2)
image = torch.transpose(image,0,1)
image = transform(image)
#image = val_set[6][0]
image = image.unsqueeze(0).cuda()

with torch.no_grad():
    probs = torch.nn.Softmax(dim=-1)(network(image))
    for j in range(1,6):
        count += 1
        ax = fig2.add_subplot(4,5, count)
        ax.set_title("Layer {}".format(j), fontsize= 30)
        plt.axis('off')
        # Channel 3 of the image.
        plt.imshow(deconv(network.feature_outputs["maxpooling_3"], network.switch_indices,j).cpu().detach().numpy()[0, 2, :])

In [ ]:
image = val_set[6][0]
import matplotlib.pyplot as plt
plt.imshow(image.permute(1,2,0))

In [ ]:
tool.test_model(network,"ZFNet",test_set, accuracy=81.22,batch_size=20,num_workers=4)

In [ ]:
categories = test_set.classes
print(categories)
Vis = Visualize(categories=categories)
val_loader = torch.utils.data.DataLoader(val_set,batch_size = 5, shuffle = True, num_workers = 2)
batch = next(iter(val_loader))
Vis.feature_visualize(network, network.conv2,"conv2",batch)